# Ray Train - A Library for Distributed Deep Learning

© 2019-2022, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

[Ray Train](https://docs.ray.io/en/latest/train/train.html) is a lightweight library for distributed deep learning. It provides thin wrappers around [PyTorch](https://pytorch.org), [TensorFlow](https://tensorflow.org), and [Horvod](https://horovod.ai/) native modules for data parallel training.

> **NOTE**: Ray SGD is renamed to Ray Train

### Introduction to Ray Train

Ray Train is a library that aims to simplify distributed deep learning. As a library, Ray Train is built to abstract away the coordination/configuration setup of distributed deep learning frameworks such as [Pytorch Distributed](https://pytorch.org/tutorials/beginner/dist_overview.html) and [Tensorflow Distributed](https://www.tensorflow.org/guide/distributed_training), allowing users to only focus on implementing training logic for their respective framework. For example: 
 * For Pytorch, Ray Train automatically handles the construction of the distributed process group.
 * For Tensorflow, Ray Train automatically handles the coordination of the `TF_CONFIG`. The current implementation assumes that the user will use a _MultiWorkerMirroredStrategy_, but this will change in the near future.
 * For Horovod, Ray Train automatically handles the construction of the Horovod runtime and [Rendezvous server](https://horovod.readthedocs.io/en/stable/_modules/horovod/ray/runner.html).

Built for data scientists/ML practitioners, Ray Train has support for standard ML tools and features that practitioners love. For example:
 * Callbacks for early stopping, reducing costs and time for training
 * Checkpointing at regular intervals, allowing to restart for fault-tolerence
 * Integration with Tensorboard, Weights/Biases, and MLflow, providing extensibilty for experimentation and observation of runs
 * Jupyter notebooks, giving developers familiar development tools for iteration and experimentation

More importantly, Ray Train integrates with the Ray Ecosystem. Distributed deep learning often comes with a lot of complexity, so you can:
 * Use [Ray Datasets](https://docs.ray.io/en/latest/data/dataset.html#datasets) with Ray Train to inject, handle or train on large amounts of data
 * Use [Ray Tune](https://docs.ray.io/en/latest/tune/index.html#tune-main) with Ray Train to leverage cutting edge hyperparameter techniques and distribute both your training and tuning
 * Use the [Ray cluster launcher](https://docs.ray.io/en/latest/cluster/cloud.html#cluster-cloud) to launch and leverage autoscaling or spot instance clusters to train your model at scale on any cloud

### Ray Train Architecture and concepts

<img src="https://docs.ray.io/en/latest/_images/train-arch.svg" width="50%" height="60%"> 

**Trainer**: The Trainer is the main class that is exposed in the [Ray Train API](https://docs.ray.io/en/latest/train/api.html) that users will interact with. A user will pass in a function which defines the training logic. In our case, the trainin
function is `train_func_distributed` with `configs` as its argument. The Trainer will create an Executor to run the distributed training. It will also will handle callbacks based on the results from the `BackendExecutor`. Read the Trainer [source here](https://github.com/ray-project/ray/blob/f1acabe9cf37d5d123017fb3f158c37fb36499a5/python/ray/train/trainer.py#L78).

**BackendExecutor**: The executor is an interface that handles execution of distributed training. It creates an actor group and initializes in conjunction with a specific backend. Worker resources, number of workers, and placement strategy are passed to the `Worker Group.` Read the BackendExecutor [source here](https://github.com/ray-project/ray/blob/f1acabe9cf37d5d123017fb3f158c37fb36499a5/python/ray/train/backend.py#L102).

**Backend**: A backend is used in conjunction with the `Executor` to initialize and manage framework-specific communication protocols. Each communication library (Torch, Horovod, TensorFlow, etc.) will have a separate backend and will take a specific configuration value. In the diagram, they are labelled as `XBackend`, `XConfig`, `YBackend`, and `YConfig` respectively. Read the Backend [source here](https://github.com/ray-project/ray/blob/f1acabe9cf37d5d123017fb3f158c37fb36499a5/python/ray/train/trainer.py#L64).

**WorkerGroup**:The `WorkerGroup` is a generic utility class for managing a group of Ray Actors, regardless of the backend. Read WorkGroup [source here](https://github.com/ray-project/ray/blob/f1acabe9cf37d5d123017fb3f158c37fb36499a5/python/ray/train/worker_group.py#L84).


### Quick Start: Distributed training on multiple workers with PyTorch

Let's work through a typical distributed PyTorch trainining example, where we only use Ray Train with multipler workper process.

In [1]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
#from tqdm.notebook import tqdm_notebook
from tqdm import tqdm

# import from Ray 
from ray import train
from ray.train import Trainer

### Step 1. Define constants, input and output variables

In [2]:
NUM_SAMPLES = 20             # our dataset for training
INPUT_SIZE = 20              # inputs or neurons into the first layer
LAYER_SIZE = 15              # inputs or neurons to the hidden layer
OUTPUT_SIZE = 5              # outputs to the last layer

# In this example we use a randomly generated dataset.
input = torch.randn(NUM_SAMPLES, INPUT_SIZE)         # In normal ML parlance, X
labels = torch.randn(NUM_SAMPLES, OUTPUT_SIZE)       # In nmormal ML parlance, y

### Step 2: Define a simple PyTorch neural network

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(in_features=INPUT_SIZE, out_features=LAYER_SIZE)
        # Our activation function
        self.relu = nn.ReLU()           
        self.layer2 = nn.Linear(in_features=LAYER_SIZE, out_features=OUTPUT_SIZE)

    def forward(self, input):
        return self.layer2(self.relu(self.layer1(input)))

### Step 3: Define our training function used by Ray Train
Simple function that iterates over epochs and does standard PyTorch steps:
 * Invoke the callable model with input
 * Calculate the loss
 * Zero out the gradients
 * Do backward propogation
 * Optimize the step

In [4]:
def train_func_distributed(config):
   
    model = NeuralNetwork()
    model = train.torch.prepare_model(model, move_to_device=True)
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)

    # Iterate over the loop
    epochs = config.get('NUM_EPOCHS', [20, 40, 60])
    for epoch in epochs: 
        for e in tqdm(range(epoch)):
            output = model(input)
            loss = loss_fn(output, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            if e % epoch == 0:
                print(f'epoch {epoch}, loss: {loss.item():.3f}')
    # Return anything you want, here we just report back the pid on which this function
    # runs
    return os.getpid()

### Step 4: Train the model

We create the Trainer, the main class as shown in the above Ray Train architecture diagram. This in turn will connect to the Ray cluster, without us using `ray.init(...)`. 

In [5]:
trainer = Trainer(backend='torch', num_workers=4)
trainer.start()
results = trainer.run(train_func_distributed, config={'NUM_EPOCHS': [20, 40, 60]})
print(results)

2022-03-16 16:13:37,747	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8266
2022-03-16 16:13:40,251	INFO trainer.py:199 -- Trainer logs will be logged in: /Users/jules/ray_results/train_2022-03-16_16-13-40
(BaseWorkerMixin pid=60843) 2022-03-16 16:13:41,866	INFO torch.py:66 -- Setting up process group for: env:// [rank=0, world_size=4]
(BaseWorkerMixin pid=60838) 2022-03-16 16:13:41,867	INFO torch.py:66 -- Setting up process group for: env:// [rank=1, world_size=4]
(BaseWorkerMixin pid=60841) 2022-03-16 16:13:41,866	INFO torch.py:66 -- Setting up process group for: env:// [rank=2, world_size=4]
(BaseWorkerMixin pid=60837) 2022-03-16 16:13:41,866	INFO torch.py:66 -- Setting up process group for: env:// [rank=3, world_size=4]
(BaseWorkerMixin pid=60843) [W ProcessGroupGloo.cpp:707] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator(

(BaseWorkerMixin pid=60843) epoch 20, loss: 1.313
(BaseWorkerMixin pid=60837) epoch 20, loss: 1.313
(BaseWorkerMixin pid=60841) epoch 20, loss: 1.313
(BaseWorkerMixin pid=60841) epoch 40, loss: 0.897
(BaseWorkerMixin pid=60838) epoch 20, loss: 1.313
(BaseWorkerMixin pid=60838) epoch 40, loss: 0.897
(BaseWorkerMixin pid=60843) epoch 40, loss: 0.897
(BaseWorkerMixin pid=60837) epoch 40, loss: 0.897
(BaseWorkerMixin pid=60843) epoch 60, loss: 0.506
(BaseWorkerMixin pid=60837) epoch 60, loss: 0.506
(BaseWorkerMixin pid=60841) epoch 60, loss: 0.506
(BaseWorkerMixin pid=60838) epoch 60, loss: 0.506
[60843, 60838, 60841, 60837]


100%|██████████| 60/60 [00:00<00:00, 926.31it/s]


### Excercises

Have a go at this in your spare time and observe the results

 1. Change the NUM_EPOCHS list to **[200, 400, 600]**
 2. Do you see the loss approaching zero?
 3. Try changing sample sizes. Do you need more epochs to train and minimize loss?
 4. Try chaning the number of workers to 1/2 number of cores on your localhost or laptop

In [6]:
trainer.shutdown()

(BaseWorkerMixin pid=60843) /usr/local/anaconda3/lib/python3.8/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(BaseWorkerMixin pid=60843)   warnings.warn('resource_tracker: There appear to be %d '
(BaseWorkerMixin pid=60838) /usr/local/anaconda3/lib/python3.8/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(BaseWorkerMixin pid=60838)   warnings.warn('resource_tracker: There appear to be %d '
(BaseWorkerMixin pid=60841) /usr/local/anaconda3/lib/python3.8/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(BaseWorkerMixin pid=60841)   warnings.warn('resource_tracker: There appear to be %d '
